In [ ]:
#default_exp merge

In [ ]:
#export
from nbprocess.imports import *
from nbprocess.read import *
from fastcore.script import *

In [ ]:
import json

# Fix merge conflicts

> Fix merge conflicts in jupyter notebooks

When working with jupyter notebooks (which are json files behind the scenes) and GitHub, it is very common that a merge conflict (that will add new lines in the notebook source file) will break some notebooks you are working on. This module defines the function `fix_conflicts` to fix those notebooks for you, and attempt to automatically merge standard conflicts. The remaining ones will be delimited by markdown cells like this:

<img alt="Fixed notebook" width="700" caption="A notebook fixed after a merged conflict. The file couldn't be opened before the command was run, but after it the conflict is highlighted by markdown cells." src="images/merge.PNG" />

## Walk cells

This is an example of broken notebook we defined in `tst_nb`. The json format is broken by the lines automatically added by git. Such a file can't be opened again in jupyter notebook, leaving the user with no other choice than to fix the text file manually.

In [ ]:
tst_nb=r"""{
 "cells": [
  {
   "cell_type": "code",
<<<<<<< HEAD
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "3"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "z=3\n",
    "z"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
=======
   "execution_count": 5,
>>>>>>> a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "6"
      ]
     },
<<<<<<< HEAD
     "execution_count": 7,
=======
     "execution_count": 5,
>>>>>>> a7ec1b0bfb8e23b05fd0a2e6cafcb41cd0fb1c35
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "x=3\n",
    "y=3\n",
    "x+y"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
"""

Note that in this example, the second conflict is easily solved: it just concerns the execution count of the second cell and can be solved by choosing either option without really impacting your notebook. This is the kind of conflicts `fix_conflicts` will (by default) fix automatically. The first conflict is more complicated as it spans across two cells and there is a cell present in one version, not the other. Such a conflict (and generally the ones where the inputs of the cells change form one version to the other) aren't automatically fixed, but we will return a proper json file where the annotations introduced by git will be placed in markdown cells.

In [ ]:
#export
_BEG,_MID,_END = '<'*7,'='*7,'>'*7
conf_re = re.compile(rf'^{_BEG}\s+(\S+)\n(.*?)\n{_MID}\n(.*?)^{_END}\s+(\S+)$', re.MULTILINE|re.DOTALL)

def _unpatch_f(before, cb1, cb2, c, r):
    if cb1 is not None and cb1 != cb2: raise Exception(f'Branch mismatch: {cb1}/{cb2}')
    r.append(before)
    r.append(c)
    return cb2

def _unpatch_res(r, last): return dict2nb(json.loads(''.join(r+[last])))

def _unpatch(s):
    conflicts = conf_re.split(s)
    r1,r2 = [],[]
    c1b,c2b = None,None
    *main,last = conflicts
    for before,c1_branch,c1,c2,c2_branch in chunked(main, 5):
        c1b = _unpatch_f(before, c1b, c1_branch, c1, r1)
        c2b = _unpatch_f(before, c2b, c2_branch, c2, r2)
    return _unpatch_res(r1, last),_unpatch_res(r2, last)

In [ ]:
a,b = _unpatch(tst_nb)

In [ ]:
from difflib import SequenceMatcher

In [ ]:
sm = SequenceMatcher(None, a.cells, b.cells)

In [ ]:
sm.get_matching_blocks()

[Match(a=1, b=0, size=2), Match(a=3, b=2, size=0)]

When walking the broken cells, we will add conflicts marker before and after the cells with conflicts as markdown cells. To do that we use this function.

In [ ]:
#export
def get_md_cell(txt):
    "A markdown cell with `txt`"
    return '''  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "%s"
   ]
  },''' % txt

In [ ]:
tst = '''  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A bit of markdown"
   ]
  },'''
assert get_md_cell("A bit of markdown") == tst

In [ ]:
#export
_re_conflict = re.compile(r'^<<<<<<<', re.MULTILINE)

In [ ]:
#hide
assert _re_conflict.search('a\nb\nc') is None
assert _re_conflict.search('a\n<<<<<<<\nc') is not None

## Main function

In [ ]:
#export
@call_parse
def nbdev_fix_merge(fname:str, # notebook filename to fix
                    trust_us:bool=True): # use our outputs/metadata instead of theirs
    "Fix merge conflicts in notebook `fname`"
    fname=Path(fname)
    shutil.copy(fname, fname.with_suffix('.ipynb.bak'))
    raw_text = fname.read_text()
    if not added: print("Successfully merged conflicts!")
    else: print("One or more conflict remains in the notebook, please inspect manually.")

This begins by backing the notebook `fname` to `fname.bak` in case something goes wrong. Then it parses the broken json, solving conflicts in cells. Every conflict that only involves metadata or outputs of cells will be solved automatically by using the local (`trust_us=True`) or the remote (`trust_us=False`) branch. Otherwise, or for conflicts involving the inputs of cells, the json will be repaired by including the two version of the conflicted cell(s) with markdown cells indicating the conflicts. You will be able to open the notebook again and search for the conflicts (look for `<<<<<<<`) then fix them as you wish.

The function will print a message indicating whether the notebook was fully merged or if conflicts remain.

## Export-

In [ ]:
#hide
from nbprocess.export import nbs_export
nbs_export()